In [8]:
import altair as alt
import json
import pandas as pd 
import os
import requests
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
FORMAT = "json"
PVWATTS_URL = f"https://developer.nrel.gov/api/pvwatts/v8.{FORMAT}"
print(f"Format of the output data: {FORMAT}")

Format of the output data: json


In [9]:
%%time

payload = {
    "api_key": API_KEY,
    "system_capacity": 100, # 100KW 
    "module_type": 0, # 0 for standard modules 
    "losses": 0, # Losses in percent 
    "array_type": 1, # Fixed roof mounted 
    "tilt": 10, # Tilt angle 
    "azimuth": 78, 
    "lat": 1.3521,
    "lon": 103.8198,
    "timeframe": "hourly"
}

response = requests.get(PVWATTS_URL, params=payload)

CPU times: user 40.2 ms, sys: 0 ns, total: 40.2 ms
Wall time: 2.34 s


In [10]:
ac = response.json()["outputs"]["ac"]

source = pd.DataFrame({
    "hour": pd.date_range(start="01/01/2018", freq="H", periods=8760*5, inclusive="both"),
    "AC Power": ac*5 
})

In [11]:
run_cell = False

if run_cell:
    with open("data/pvwatts/1year.json", "w") as outfile:
        json.dump(response.json(), outfile)

In [13]:
alt.data_transformers.disable_max_rows()


alt.Chart(source).mark_line().encode(
    x="hour",
    y="AC Power"
).properties(width=1000, height=600)

alt.Chart(...)

In [ ]:
run_cell = False

if run_cell:
    source.to_csv("data/pvwatts/5years_baseline.csv", index=False)